In [26]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_wy.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,966418,2724797100,4/11/20,897,PPP,EAGLE SPECIALTY MATERIALS LLC,PO Box 3039,GILLETTE,WY,82718.0,...,2123 Nonmetallic Mineral Mining and Quarrying,2123 Nonmetallic Mineral Mining and Quarrying,2123.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,NaN
1,966419,9692597008,4/9/20,897,PPP,TRIHYDRO CORPORATION,1252 Commerce Dr,LARAMIE,WY,82070.0,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,NaN
2,966420,6083237006,4/6/20,897,PPP,"CYCLONE DRILLING, INC.",5800 Mohan Road PO Box 908,GILLETTE,WY,82717.0,...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,NaN
3,966421,7125427008,4/7/20,897,PPP,"REDI SERVICES, LLC",225 W OWEN ST,LYMAN,WY,82937.0,...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,NaN
4,966422,8862647107,4/15/20,897,PPP,MEMORIAL HOSPITAL OF CONVERSE COUNTY,700 East CENTER ST,DOUGLAS,WY,82633.0,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,968552,4395967002,4/3/20,897,PPP,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,CASPER,WY,82601.0,...,999 Other,6212 Offices of Dentists,6212.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"102 N. Kenwood, CASPER, WY",42.850300,-106.304365,NaN
2135,968553,6985647108,4/14/20,897,PPP,"SWEETWATER COUNTY CHILD DEVELOPMENTAL CENTER, ...",1715 HITCHING POST DR,GREEN RIVER,WY,82935.0,...,999 Other,6244 Child Day Care Services,6244.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1715 HITCHING POST DR, GREEN RIVER, WY",41.499071,-109.464675,NaN
2136,968554,7996438405,2/12/21,897,PPS,ELECTRICAL SYSTEMS OF WYOMING INC,1105 Adon Rd,Rozet,WY,82727.0,...,999 Other,2382 Building Equipment Contractors,2382.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1105 Adon Rd, Rozet, WY",44.435381,-105.226008,NaN
2137,968555,9054647103,4/15/20,897,PPP,EDEN LIFE CARE,30 N. Gould Street Suite 4000,SHERIDAN,WY,82801.0,...,999 Other,6216 Home Health Care Services,6216.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"30 N. Gould Street Suite 4000, SHERIDAN, WY",44.797763,-106.954952,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,2724797100,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221
1,9692597008,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334
2,6083237006,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612
3,7125427008,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091
4,8862647107,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [27]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/WY_fips_scraped.csv") 
fips_check_scraped.head() #15869

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,966418,966418,966418,2724797100,4/11/20,897,PPP,EAGLE SPECIALTY MATERIALS LLC,PO Box 3039,...,2123 Nonmetallic Mineral Mining and Quarrying,2123 Nonmetallic Mineral Mining and Quarrying,2123.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14
1,1,966419,966419,966419,9692597008,4/9/20,897,PPP,TRIHYDRO CORPORATION,1252 Commerce Dr,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14
2,2,966420,966420,966420,6083237006,4/6/20,897,PPP,"CYCLONE DRILLING, INC.",5800 Mohan Road PO Box 908,...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14
3,3,966421,966421,966421,7125427008,4/7/20,897,PPP,"REDI SERVICES, LLC",225 W OWEN ST,...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14
4,4,966422,966422,966422,8862647107,4/15/20,897,PPP,MEMORIAL HOSPITAL OF CONVERSE COUNTY,700 East CENTER ST,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14


In [28]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [29]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [30]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,2724797100,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14
1,9692597008,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14
2,6083237006,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14
3,7125427008,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14
4,8862647107,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14
...,...,...,...,...,...
2134,4395967002,"102 N. Kenwood, CASPER, WY",42.850300,-106.304365,NaN
2135,6985647108,"1715 HITCHING POST DR, GREEN RIVER, WY",41.499071,-109.464675,NaN
2136,7996438405,"1105 Adon Rd, Rozet, WY",44.435381,-105.226008,NaN
2137,9054647103,"30 N. Gould Street Suite 4000, SHERIDAN, WY",44.797763,-106.954952,NaN


In [31]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

,LoanNumber,full_add,Lat,Long,FIPS_z
721,6427467010,"3330 I-80 Rd, CHEYENNE, WY",40.822471,-97.167532,NaN
1378,5725807009,"119 E BRIDGE ST, SARATOGA, WY",39.986975,-104.821733,NaN
1448,9082927003,"4748 Highway 89 N, EVANSTON, WY",41.646038,-111.913004,NaN
1729,6664317004,"401 Crescent Ave., MILLS, WY",39.225801,-84.465613,NaN
1905,5288408603,"1401 E Hwy 14-16, Upton, WY",44.326301,-94.473850,NaN
...,...,...,...,...,...
2134,4395967002,"102 N. Kenwood, CASPER, WY",42.850300,-106.304365,NaN
2135,6985647108,"1715 HITCHING POST DR, GREEN RIVER, WY",41.499071,-109.464675,NaN
2136,7996438405,"1105 Adon Rd, Rozet, WY",44.435381,-105.226008,NaN
2137,9054647103,"30 N. Gould Street Suite 4000, SHERIDAN, WY",44.797763,-106.954952,NaN


<h4>FIPS Script

In [32]:
#if geocode file
#lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
#long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [33]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
executable_path = {'executable_path': ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', **executable_path, options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('56')] #WY
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/157 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [34]:
geo_id_list =geo_id 

Length Check

In [35]:
print(len(geo_id_list))

157


Matching FIPS to DataFrame

In [36]:
#for geocode file
#fips_fill = fips_check_notna.iloc[:1987] #confirm pair to notna DataFrame
#if FIPS available
fips_fill = fips_check_scraped_null

In [37]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
721,6427467010,"3330 I-80 Rd, CHEYENNE, WY",40.822471,-97.167532,[]
1378,5725807009,"119 E BRIDGE ST, SARATOGA, WY",39.986975,-104.821733,[]
1448,9082927003,"4748 Highway 89 N, EVANSTON, WY",41.646038,-111.913004,[]
1729,6664317004,"401 Crescent Ave., MILLS, WY",39.225801,-84.465613,[]
1905,5288408603,"1401 E Hwy 14-16, Upton, WY",44.326301,-94.473850,[]
...,...,...,...,...,...
2134,4395967002,"102 N. Kenwood, CASPER, WY",42.850300,-106.304365,[560250003001012]
2135,6985647108,"1715 HITCHING POST DR, GREEN RIVER, WY",41.499071,-109.464675,[560379707001020]
2136,7996438405,"1105 Adon Rd, Rozet, WY",44.435381,-105.226008,[560050007023086]
2137,9054647103,"30 N. Gould Street Suite 4000, SHERIDAN, WY",44.797763,-106.954952,[560330003001053]


Remove brackets

In [38]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LoanNumber,full_add,Lat,Long,FIPS_z
721,6427467010,"3330 I-80 Rd, CHEYENNE, WY",40.822471,-97.167532,NaN
1378,5725807009,"119 E BRIDGE ST, SARATOGA, WY",39.986975,-104.821733,NaN
1448,9082927003,"4748 Highway 89 N, EVANSTON, WY",41.646038,-111.913004,NaN
1729,6664317004,"401 Crescent Ave., MILLS, WY",39.225801,-84.465613,NaN
1905,5288408603,"1401 E Hwy 14-16, Upton, WY",44.326301,-94.473850,NaN
...,...,...,...,...,...
2134,4395967002,"102 N. Kenwood, CASPER, WY",42.850300,-106.304365,560250003001012
2135,6985647108,"1715 HITCHING POST DR, GREEN RIVER, WY",41.499071,-109.464675,560379707001020
2136,7996438405,"1105 Adon Rd, Rozet, WY",44.435381,-105.226008,560050007023086
2137,9054647103,"30 N. Gould Street Suite 4000, SHERIDAN, WY",44.797763,-106.954952,560330003001053


In [39]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
721,6427467010,"3330 I-80 Rd, CHEYENNE, WY",40.822471,-97.167532,NaN
1378,5725807009,"119 E BRIDGE ST, SARATOGA, WY",39.986975,-104.821733,NaN
1448,9082927003,"4748 Highway 89 N, EVANSTON, WY",41.646038,-111.913004,NaN
1729,6664317004,"401 Crescent Ave., MILLS, WY",39.225801,-84.465613,NaN
1905,5288408603,"1401 E Hwy 14-16, Upton, WY",44.326301,-94.473850,NaN


Parse for Merge

In [40]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
721,6427467010,NaN
1378,5725807009,NaN
1448,9082927003,NaN
1729,6664317004,NaN
1905,5288408603,NaN
...,...,...
2134,4395967002,560250003001012
2135,6985647108,560379707001020
2136,7996438405,560050007023086
2137,9054647103,560330003001053


Read in base data

In [41]:
#if from geocode file
#fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_wy.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
#fips_check.head()

#if from FIPS file
fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/WY_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,966418,966418,966418,2724797100,4/11/20,897,PPP,EAGLE SPECIALTY MATERIALS LLC,PO Box 3039,...,2123 Nonmetallic Mineral Mining and Quarrying,2123 Nonmetallic Mineral Mining and Quarrying,2123.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14
1,1,966419,966419,966419,9692597008,4/9/20,897,PPP,TRIHYDRO CORPORATION,1252 Commerce Dr,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14
2,2,966420,966420,966420,6083237006,4/6/20,897,PPP,"CYCLONE DRILLING, INC.",5800 Mohan Road PO Box 908,...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14
3,3,966421,966421,966421,7125427008,4/7/20,897,PPP,"REDI SERVICES, LLC",225 W OWEN ST,...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14
4,4,966422,966422,966422,8862647107,4/15/20,897,PPP,MEMORIAL HOSPITAL OF CONVERSE COUNTY,700 East CENTER ST,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14


Merge with MainFrame

In [42]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,0,966418,966418,966418,2724797100,4/11/20,897,PPP,EAGLE SPECIALTY MATERIALS LLC,PO Box 3039,...,2123 Nonmetallic Mineral Mining and Quarrying,2123.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14,NaN
1,1,966419,966419,966419,9692597008,4/9/20,897,PPP,TRIHYDRO CORPORATION,1252 Commerce Dr,...,"5413 Architectural, Engineering, and Related S...",5413.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14,NaN
2,2,966420,966420,966420,6083237006,4/6/20,897,PPP,"CYCLONE DRILLING, INC.",5800 Mohan Road PO Box 908,...,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14,NaN
3,3,966421,966421,966421,7125427008,4/7/20,897,PPP,"REDI SERVICES, LLC",225 W OWEN ST,...,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14,NaN
4,4,966422,966422,966422,8862647107,4/15/20,897,PPP,MEMORIAL HOSPITAL OF CONVERSE COUNTY,700 East CENTER ST,...,6221 General Medical and Surgical Hospitals,6221.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,2134,968552,968552,968552,4395967002,4/3/20,897,PPP,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,...,6212 Offices of Dentists,6212.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"102 N. Kenwood, CASPER, WY",42.850300,-106.304365,NaN,560250003001012
2135,2135,968553,968553,968553,6985647108,4/14/20,897,PPP,"SWEETWATER COUNTY CHILD DEVELOPMENTAL CENTER, ...",1715 HITCHING POST DR,...,6244 Child Day Care Services,6244.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1715 HITCHING POST DR, GREEN RIVER, WY",41.499071,-109.464675,NaN,560379707001020
2136,2136,968554,968554,968554,7996438405,2/12/21,897,PPS,ELECTRICAL SYSTEMS OF WYOMING INC,1105 Adon Rd,...,2382 Building Equipment Contractors,2382.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1105 Adon Rd, Rozet, WY",44.435381,-105.226008,NaN,560050007023086
2137,2137,968555,968555,968555,9054647103,4/15/20,897,PPP,EDEN LIFE CARE,30 N. Gould Street Suite 4000,...,6216 Home Health Care Services,6216.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"30 N. Gould Street Suite 4000, SHERIDAN, WY",44.797763,-106.954952,NaN,560330003001053


<b>Fill FIPS Columns - if FIPS file

In [43]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,966418,966418,966418,2724797100,4/11/20,897,PPP,EAGLE SPECIALTY MATERIALS LLC,PO Box 3039,...,2123.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14,NaN,560050004002053.0
1,1,966419,966419,966419,9692597008,4/9/20,897,PPP,TRIHYDRO CORPORATION,1252 Commerce Dr,...,5413.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14,NaN,560019627001000.0
2,2,966420,966420,966420,6083237006,4/6/20,897,PPP,"CYCLONE DRILLING, INC.",5800 Mohan Road PO Box 908,...,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14,NaN,560050002004061.0
3,3,966421,966421,966421,7125427008,4/7/20,897,PPP,"REDI SERVICES, LLC",225 W OWEN ST,...,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14,NaN,560419752012022.0
4,4,966422,966422,966422,8862647107,4/15/20,897,PPP,MEMORIAL HOSPITAL OF CONVERSE COUNTY,700 East CENTER ST,...,6221.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14,NaN,560099564001050.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,2134,968552,968552,968552,4395967002,4/3/20,897,PPP,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,...,6212.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"102 N. Kenwood, CASPER, WY",42.850300,-106.304365,NaN,560250003001012,560250003001012
2135,2135,968553,968553,968553,6985647108,4/14/20,897,PPP,"SWEETWATER COUNTY CHILD DEVELOPMENTAL CENTER, ...",1715 HITCHING POST DR,...,6244.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1715 HITCHING POST DR, GREEN RIVER, WY",41.499071,-109.464675,NaN,560379707001020,560379707001020
2136,2136,968554,968554,968554,7996438405,2/12/21,897,PPS,ELECTRICAL SYSTEMS OF WYOMING INC,1105 Adon Rd,...,2382.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1105 Adon Rd, Rozet, WY",44.435381,-105.226008,NaN,560050007023086,560050007023086
2137,2137,968555,968555,968555,9054647103,4/15/20,897,PPP,EDEN LIFE CARE,30 N. Gould Street Suite 4000,...,6216.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"30 N. Gould Street Suite 4000, SHERIDAN, WY",44.797763,-106.954952,NaN,560330003001053,560330003001053


Drop Extra FIPS Columns

In [44]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,966418,966418,966418,2724797100,4/11/20,897,PPP,EAGLE SPECIALTY MATERIALS LLC,PO Box 3039,...,2123 Nonmetallic Mineral Mining and Quarrying,2123 Nonmetallic Mineral Mining and Quarrying,2123.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,560050004002053.0
1,1,966419,966419,966419,9692597008,4/9/20,897,PPP,TRIHYDRO CORPORATION,1252 Commerce Dr,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,560019627001000.0
2,2,966420,966420,966420,6083237006,4/6/20,897,PPP,"CYCLONE DRILLING, INC.",5800 Mohan Road PO Box 908,...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,560050002004061.0
3,3,966421,966421,966421,7125427008,4/7/20,897,PPP,"REDI SERVICES, LLC",225 W OWEN ST,...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,560419752012022.0
4,4,966422,966422,966422,8862647107,4/15/20,897,PPP,MEMORIAL HOSPITAL OF CONVERSE COUNTY,700 East CENTER ST,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,560099564001050.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,2134,968552,968552,968552,4395967002,4/3/20,897,PPP,"ROY E PAULSON, JR., P.C.",102 N. Kenwood,...,999 Other,6212 Offices of Dentists,6212.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"102 N. Kenwood, CASPER, WY",42.850300,-106.304365,560250003001012
2135,2135,968553,968553,968553,6985647108,4/14/20,897,PPP,"SWEETWATER COUNTY CHILD DEVELOPMENTAL CENTER, ...",1715 HITCHING POST DR,...,999 Other,6244 Child Day Care Services,6244.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1715 HITCHING POST DR, GREEN RIVER, WY",41.499071,-109.464675,560379707001020
2136,2136,968554,968554,968554,7996438405,2/12/21,897,PPS,ELECTRICAL SYSTEMS OF WYOMING INC,1105 Adon Rd,...,999 Other,2382 Building Equipment Contractors,2382.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"1105 Adon Rd, Rozet, WY",44.435381,-105.226008,560050007023086
2137,2137,968555,968555,968555,9054647103,4/15/20,897,PPP,EDEN LIFE CARE,30 N. Gould Street Suite 4000,...,999 Other,6216 Home Health Care Services,6216.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"30 N. Gould Street Suite 4000, SHERIDAN, WY",44.797763,-106.954952,560330003001053


In [45]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
721,721,967139,967139,967139,6427467010,4/6/20,897,PPP,BAR S SERVICES INC.,3330 I-80 Rd,...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"3330 I-80 Rd, CHEYENNE, WY",40.822471,-97.167532,NaN
1378,1378,967796,967796,967796,5725807009,4/6/20,897,PPP,SHIVELY HARDWARE COMPANY,119 E BRIDGE ST,...,999 Other,4441 Building Material and Supplies Dealers,4441.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"119 E BRIDGE ST, SARATOGA, WY",39.986975,-104.821733,NaN
1448,1448,967866,967866,967866,9082927003,4/9/20,897,PPP,"WYOMING FABRICATION ENTERPRISES, INC.",4748 Highway 89 N,...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4748 Highway 89 N, EVANSTON, WY",41.646038,-111.913004,NaN
1729,1729,968147,968147,968147,6664317004,4/7/20,897,PPP,"INSTALLATION & SERVICE CO., INC.",401 Crescent Ave.,...,999 Other,2371 Utility System Construction,2371.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"401 Crescent Ave., MILLS, WY",39.225801,-84.465613,NaN
1905,1905,968323,968323,968323,5288408603,3/20/21,897,PPS,WESTON ENGINEERING INC,1401 E Hwy 14-16,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1401 E Hwy 14-16, Upton, WY",44.326301,-94.473850,NaN


Convert to File

In [46]:
fips_WY = fips_merge_drop

In [47]:
fips_WY.to_csv("../../data/state_data/geo/geo_fips/WY_fips_scraped.csv") 

Review & Compare

In [48]:
geo_wy_fips = pd.read_csv("../../data/state_data/geo/geo_fips/WY_fips_scraped.csv") 
geo_wy_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,966418,966418,966418,2724797100,4/11/20,897,PPP,EAGLE SPECIALTY MATERIALS LLC,...,2123 Nonmetallic Mineral Mining and Quarrying,2123 Nonmetallic Mineral Mining and Quarrying,2123.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"PO Box 3039, GILLETTE, WY",44.291092,-105.502221,5.600500e+14
1,1,1,966419,966419,966419,9692597008,4/9/20,897,PPP,TRIHYDRO CORPORATION,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"1252 Commerce Dr, LARAMIE, WY",41.299319,-105.611334,5.600196e+14
2,2,2,966420,966420,966420,6083237006,4/6/20,897,PPP,"CYCLONE DRILLING, INC.",...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"5800 Mohan Road PO Box 908, GILLETTE, WY",44.241265,-105.474612,5.600500e+14
3,3,3,966421,966421,966421,7125427008,4/7/20,897,PPP,"REDI SERVICES, LLC",...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"225 W OWEN ST, LYMAN, WY",41.327013,-110.297091,5.604198e+14
4,4,4,966422,966422,966422,8862647107,4/15/20,897,PPP,MEMORIAL HOSPITAL OF CONVERSE COUNTY,...,999 Other,6221 General Medical and Surgical Hospitals,6221.0,WYOMING DISTRICT OFFICE,e $5-10 million,250 to 499,"700 East CENTER ST, DOUGLAS, WY",42.760407,-105.378531,5.600996e+14


In [49]:
##### fill in missing Lat, Long, FIPS
geo_wy_fips.loc[geo_wy_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
721,721,721,967139,967139,967139,6427467010,4/6/20,897,PPP,BAR S SERVICES INC.,...,999 Other,2131 Support Activities for Mining,2131.0,WYOMING DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"3330 I-80 Rd, CHEYENNE, WY",40.822471,-97.167532,NaN
1378,1378,1378,967796,967796,967796,5725807009,4/6/20,897,PPP,SHIVELY HARDWARE COMPANY,...,999 Other,4441 Building Material and Supplies Dealers,4441.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"119 E BRIDGE ST, SARATOGA, WY",39.986975,-104.821733,NaN
1448,1448,1448,967866,967866,967866,9082927003,4/9/20,897,PPP,"WYOMING FABRICATION ENTERPRISES, INC.",...,332 Fabricated Metal Products,3329 Other Fabricated Metal Product Manufacturing,3329.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"4748 Highway 89 N, EVANSTON, WY",41.646038,-111.913004,NaN
1729,1729,1729,968147,968147,968147,6664317004,4/7/20,897,PPP,"INSTALLATION & SERVICE CO., INC.",...,999 Other,2371 Utility System Construction,2371.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"401 Crescent Ave., MILLS, WY",39.225801,-84.465613,NaN
1905,1905,1905,968323,968323,968323,5288408603,3/20/21,897,PPS,WESTON ENGINEERING INC,...,999 Other,"5413 Architectural, Engineering, and Related S...",5413.0,WYOMING DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1401 E Hwy 14-16, Upton, WY",44.326301,-94.473850,NaN
